Entrenar un modelo T5-Sentinel bajo el paradigma Seq2Seq es un reto técnico excelente para tu TFG

# 1. Preparacion del entorno y datos
T5 espera que las etiquetas sean texto. Vamos a mapear tus is_real a los targets del paper:
0 (IA) $\rightarrow$ "positive"
1 (Humano) $\rightarrow$ "negative"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
from datasets import Dataset, DatasetDict

def prepare_grouped_dataset(jsonl_path, train_size=0.8, val_size=0.1):
    df = pd.read_json(jsonl_path, lines=True)

    # Mapeo senior: IA -> positive, Humano -> negative
    df['target_text'] = df['is_real'].map({0: "positive", 1: "negative"})
    df['input_text'] = "classify authenticity: " + df['title'] + " " + df['title']

    # 1. Separar Test set (Hold-out)
    gs_test = GroupShuffleSplit(n_splits=1, train_size=train_size + val_size, random_state=42)
    train_val_idx, test_idx = next(gs_test.split(df, groups=df['group_id']))

    df_train_val = df.iloc[train_val_idx]
    df_test = df.iloc[test_idx]

    # 2. Separar Train y Eval del resto
    relative_train_size = train_size / (train_size + val_size)
    gs_eval = GroupShuffleSplit(n_splits=1, train_size=relative_train_size, random_state=42)
    train_idx, eval_idx = next(gs_eval.split(df_train_val, groups=df_train_val['group_id']))

    df_train = df_train_val.iloc[train_idx]
    df_eval = df_train_val.iloc[eval_idx]

    ds_dict = DatasetDict({
        'train': Dataset.from_pandas(df_train),
        'validation': Dataset.from_pandas(df_eval),
        'test': Dataset.from_pandas(df_test)
    })

    return ds_dict

In [11]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer, DataCollatorForSeq2Seq

def train_t5_sentinel(dataset_dict, model_name="google-t5/t5-base", config=None):
    if config is None:
        config = {
            "lr": 5e-4,
            "epochs": 5,
            "batch_size": 8,
            "grad_acc": 64,
            "weight_decay": 0.01
        }

    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    def tokenize_fn(examples):
        inputs = tokenizer(examples["input_text"], max_length=512, truncation=True, padding="max_length")
        targets = tokenizer(text_target=examples["target_text"], max_length=10, truncation=True, padding="max_length")
        inputs["labels"] = targets["input_ids"]
        return inputs

    tokenized_ds = dataset_dict.map(tokenize_fn, batched=True)

    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

    training_args = TrainingArguments(
        output_dir=f"./results_{model_name.split('/')[-1]}",
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=config["lr"],
        per_device_train_batch_size=config["batch_size"],
        per_device_eval_batch_size=config["batch_size"],
        gradient_accumulation_steps=config["grad_acc"],
        num_train_epochs=config["epochs"],
        weight_decay=config["weight_decay"],
        lr_scheduler_type="cosine",
        fp16=True, # Importante para la T4 de Colab
        push_to_hub=False,
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_ds["train"],
        eval_dataset=tokenized_ds["validation"],
        data_collator=data_collator
    )

    return trainer, model, tokenizer, tokenized_ds

In [12]:
ds_dict = prepare_grouped_dataset("/content/drive/MyDrive/TFG/titles_data.jsonl")
trainer, model, tokenizer, tokenized_dataset = train_t5_sentinel(ds_dict)# Lanzar entrenamiento
trainer.train()

# Evaluación con restricción de vocabulario
import torch

def get_sentinel_prediction(text, model, tokenizer):
    input_ids = tokenizer("classify authenticity: " + text, return_tensors="pt").input_ids.to("cuda")

    # Obtenemos los logits del primer token generado
    with torch.no_grad():
        outputs = model(input_ids=input_ids, decoder_input_ids=torch.tensor([[tokenizer.pad_token_id]]).to("cuda"))
        logits = outputs.logits[:, 0, :] # Logits del primer token de salida

    # Filtramos solo los IDs de "positive" y "negative"
    pos_id = tokenizer.encode("positive")[0]
    neg_id = tokenizer.encode("negative")[0]

    # Comparamos probabilidades relativas
    probs = torch.softmax(logits[:, [pos_id, neg_id]], dim=-1)
    return "IA" if probs[0][0] > probs[0][1] else "Humano"

Loading weights:   0%|          | 0/257 [00:00<?, ?it/s]

Map:   0%|          | 0/1680 [00:00<?, ? examples/s]

Map:   0%|          | 0/210 [00:00<?, ? examples/s]

Map:   0%|          | 0/210 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,No log,13.475163
2,No log,0.638499
3,No log,0.062431
4,No log,0.036271
5,No log,0.030385


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def evaluate_model(model, tokenizer, dataset):
    predictions = []
    true_labels = []

    # Mapeo inverso para comparar con los resultados de get_sentinel_prediction
    # is_real = 0 (IA) -> target_text = 'positive'
    # is_real = 1 (Humano) -> target_text = 'negative'
    # get_sentinel_prediction: 'IA' (for positive) o 'Humano' (for negative)

    # Es decir, si get_sentinel_prediction devuelve 'IA', esperamos 'positive'.
    # Si devuelve 'Humano', esperamos 'negative'.

    for example in dataset:
        # Asumimos que 'input_text' ya tiene el prefijo 'classify authenticity: '
        # y que 'target_text' es 'positive' o 'negative'

        # Obtener la predicción del modelo
        predicted_label = get_sentinel_prediction(example['input_text'].replace('classify authenticity: ', ''), model, tokenizer)
        predictions.append(predicted_label)
        true_labels.append(example['target_text'])

    # Para calcular las métricas, necesitamos que las etiquetas sean consistentes.
    # Mapearemos 'positive' -> 'IA' y 'negative' -> 'Humano' para que coincidan con la salida del modelo.
    # O mapearemos 'IA' -> 'positive' y 'Humano' -> 'negative' para que coincidan con el target_text.
    # Optaremos por el segundo para que las métricas se interpreten con 'positive'/'negative'

    mapped_predictions = ["positive" if p == "IA" else "negative" for p in predictions]

    print("\n--- Resultados de Evaluación ---")
    print(f"Total de ejemplos en el conjunto de prueba: {len(true_labels)}")

    # Calcular exactitud
    accuracy = accuracy_score(true_labels, mapped_predictions)
    print(f"Accuracy: {accuracy:.4f}")

    # Calcular precisión, recall, f1-score por clase
    # labels=['positive', 'negative'] asegura el orden y las etiquetas
    precision, recall, f1, _ = precision_recall_fscore_support(
        true_labels, mapped_predictions, average=None, labels=['positive', 'negative'], zero_division=0
    )

    print("\nMétricas por clase:")
    print(f"  Clase 'positive' (IA):")
    print(f"    Precision: {precision[0]:.4f}")
    print(f"    Recall:    {recall[0]:.4f}")
    print(f"    F1-score:  {f1[0]:.4f}")

    print(f"  Clase 'negative' (Humano):")
    print(f"    Precision: {precision[1]:.4f}")
    print(f"    Recall:    {recall[1]:.4f}")
    print(f"    F1-score:  {f1[1]:.4f}")

    # Calcular métricas macro y weighted (para desbalanceo de clases)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        true_labels, mapped_predictions, average='macro', zero_division=0
    )
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(
        true_labels, mapped_predictions, average='weighted', zero_division=0
    )

    print("\nMétricas promedio:")
    print(f"  Macro Precision: {precision_macro:.4f}")
    print(f"  Macro Recall:    {recall_macro:.4f}")
    print(f"  Macro F1-score:  {f1_macro:.4f}")
    print(f"  Weighted Precision: {precision_weighted:.4f}")
    print(f"  Weighted Recall:    {recall_weighted:.4f}")
    print(f"  Weighted F1-score:  {f1_weighted:.4f}")


# Ejecutar la evaluación
evaluate_model(model, tokenizer, tokenized_dataset["test"])


--- Resultados de Evaluación ---
Total de ejemplos en el conjunto de prueba: 210
Accuracy: 0.8571

Métricas por clase:
  Clase 'positive' (IA):
    Precision: 0.7907
    Recall:    0.9714
    F1-score:  0.8718
  Clase 'negative' (Humano):
    Precision: 0.9630
    Recall:    0.7429
    F1-score:  0.8387

Métricas promedio:
  Macro Precision: 0.8768
  Macro Recall:    0.8571
  Macro F1-score:  0.8553
  Weighted Precision: 0.8768
  Weighted Recall:    0.8571
  Weighted F1-score:  0.8553


#### 1. Total de ejemplos en el conjunto de prueba: 210
Este es el número total de ejemplos que el modelo utilizó para ser evaluado. Es importante tenerlo en cuenta para saber la base sobre la que se calculan las métricas.
#### 2. Accuracy: 0.8571
Exactitud (Accuracy): Indica que el 85.71% de las predicciones del modelo fueron correctas en general. Es la proporción de predicciones correctas sobre el total de predicciones.
#### 3. Métricas por clase:
Estas métricas son cruciales para entender el rendimiento del modelo en cada una de las clases ('positive' para IA y 'negative' para Humano), especialmente si las clases no están balanceadas.

##### Clase 'positive' (IA):

- Precision: 0.7907: De todos los ejemplos que el modelo predijo como 'positive' (IA), el 79.07% fueron realmente IA. Esto significa que cuando el modelo dice que algo es IA, es correcto la mayor parte del tiempo, pero no siempre.
- Recall: 0.9714: De todos los ejemplos que eran realmente 'positive' (IA), el modelo identificó correctamente el 97.14%. Esto es muy bueno; el modelo es excelente para encontrar todos los textos generados por IA.
- F1-score: 0.8718: Es la media armónica de la precisión y el recall. Es una buena métrica para equilibrar ambas. Un valor alto aquí indica un buen balance entre ser preciso y ser capaz de encontrar todos los casos de IA.
##### Clase 'negative' (Humano):

- Precision: 0.9630: De todos los ejemplos que el modelo predijo como 'negative' (Humano), el 96.30% fueron realmente humanos. Esto indica que cuando el modelo clasifica un texto como humano, es muy fiable.
- Recall: 0.7429: De todos los ejemplos que eran realmente 'negative' (Humano), el modelo identificó correctamente el 74.29%. Esto es más bajo que el recall para la clase 'positive'. Significa que el modelo tiene más dificultades para identificar todos los textos humanos reales, y puede clasificar algunos textos humanos como IA.
- F1-score: 0.8387: Similar al anterior, pero para la clase 'negative'. Refleja un buen rendimiento, aunque ligeramente inferior al de la clase 'positive', debido al recall más bajo.

#### 4. Métricas promedio:
Estas métricas resumen el rendimiento general del modelo en todas las clases.

- Macro Precision, Recall, F1-score: Calculan la media simple de las métricas por clase. No tienen en cuenta el desbalanceo de clases. Si las clases tuvieran un rendimiento muy diferente, la macro-F1 sería sensible a ello.

  - Macro Precision: 0.8768
  - Macro Recall: 0.8571
  - Macro F1-score: 0.8553
- Weighted Precision, Recall, F1-score: Calculan la media de las métricas por clase, ponderando cada clase por el número de ejemplos en esa clase. Son más útiles cuando hay un desbalanceo de clases, ya que dan más peso a la clase mayoritaria.

- Weighted Precision: 0.8768
- Weighted Recall: 0.8571
- Weighted F1-score: 0.8553


Conclusión de los resultados:
El modelo tiene una exactitud general del 85.71%, lo cual es un buen punto de partida.
Es muy bueno detectando textos generados por IA (recall de 0.9714 para 'positive'), lo que significa que rara vez se le escapa un texto de IA.
Sin embargo, tiene una precisión ligeramente menor cuando predice IA (0.7907), lo que sugiere que a veces clasifica erróneamente textos humanos como IA (falsos positivos de IA).
Cuando el modelo clasifica un texto como humano, es muy preciso (0.9630), pero tiene una tasa de falsos negativos más alta (recall de 0.7429), es decir, es más probable que clasifique un texto humano como IA que al revés.